In [2]:
pip install --upgrade google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 10.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datetime import datetime
import dlt
from google.cloud import bigquery

In [5]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-01.parquet', engine="pyarrow")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23159064 entries, 0 to 23159063
Data columns (total 7 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   dispatching_base_num    object        
 1   pickup_datetime         datetime64[ns]
 2   dropOff_datetime        datetime64[ns]
 3   PUlocationID            float64       
 4   DOlocationID            float64       
 5   SR_Flag                 float64       
 6   Affiliated_base_number  object        
dtypes: datetime64[ns](2), float64(3), object(2)
memory usage: 1.2+ GB


In [7]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,NaN,B00001
1,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,NaN,B00001
2,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,NaN,B00001
3,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,NaN,B00008
4,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,NaN,B00008


In [17]:
df["PUlocationID"].unique()

array([ nan, 140., 141., 237., 162., 145., 261.,  13., 249., 236., 171.,
       252., 265., 229.,  16., 143., 142.,  88.,  87., 206., 221., 245.,
       181., 257., 195., 138.,  25.,  89.,  70.,   7., 173.,  82.,  71.,
       188.,  72., 129.,  56., 161.,  79., 234., 107., 137., 239., 130.,
       189.,  32.,  81.,  51., 254., 264., 232.,  92.,  53., 192.,  15.,
       132., 175., 121., 135., 156., 187.,  90.,  48., 112.,  18., 127.,
       243., 144., 224., 263., 210., 147., 126., 168., 223.,  41., 119.,
       205.,  38., 255.,  37.,  17., 163., 172., 251., 233., 260., 211.,
        43., 102.,  36.,  76., 122., 159., 213., 226., 179., 167.,  69.,
        47., 148.,  78., 235., 259., 231.,  68.,  61.,  49., 106.,  66.,
       146., 174.,   3., 225.,  94., 191.,  35.,  29.,  97., 182., 170.,
       247.,  33.,  65., 256., 176.,  50., 228.,  93.,  80.,  40., 164.,
       113., 183., 134.,  95., 196.,  45., 244., 136., 198.,  62.,  52.,
        85.,  14., 116., 131.,  22., 222., 185., 15

In [8]:
urls = ['https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-01.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-02.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-03.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-04.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-05.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-06.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-07.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-08.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-09.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-10.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-11.parquet',
        'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-12.parquet',
       ]

In [4]:
def load_csv(year, service):
    if service == 'fhv':
        taxi_dtypes = {
            'dispatching_base_num': str,
            'PUlocationID': pd.Int64Dtype(),
            'DOlocationID': pd.Int64Dtype(),
            'SR_Flag': pd.Int64Dtype(),
            'Affiliated_base_number': str,
        }
    else:
        taxi_dtypes = {
            'VendorID': pd.Int64Dtype(),
            'passenger_count': pd.Int64Dtype(),
            'trip_distance': float,
            'RatecodeID': pd.Int64Dtype(),
            'store_and_fwd_flag': str,
            'PULocationID': pd.Int64Dtype(),
            'DOLocationID': pd.Int64Dtype(),
            'payment_type': pd.Int64Dtype(),
            'fare_amount': float,
            'extra': float,
            'mta_tax': float,
            'tip_amount': float,
            'tolls_amount': float,
            'improvement_surcharge': float,
            'total_amount': float,
            'congestion_surcharge': float ,
            'trip_type': pd.Int64Dtype()
        }

    if service == 'green':
        parse_dates = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']
    elif service == 'yellow':
        parse_dates = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
    else:
        parse_dates = ['pickup_datetime', 'dropOff_datetime']

    for i in range(12):
            
        # sets the month part of the file_name string
        month = '0'+str(i+1)
        month = month[-2:]

        # csv file_name
        file_name = f"{service}_tripdata_{year}-{month}.csv.gz"

        file_url = f"{init_url}{service}/{file_name}"

        for row in pd.read_csv(file_url, sep=",", compression="gzip", dtype=taxi_dtypes, parse_dates=parse_dates, iterator=True):
            yield row


In [9]:
def web_to_gcs(year, service):
    # define the connection to load to.
    generators_pipeline = dlt.pipeline(destination='bigquery', dataset_name='trips_data_all')


    # we can load any generator to a table at the pipeline destnation as follows:
    info = generators_pipeline.run(load_csv(year, service),
                                            table_name=f"{service}_tripdata",
                                            write_disposition="append")
    
    # the outcome metadata is returned by the load and we can inspect it by printing it.
    print(info)

    # Construct a BigQuery client object.
    #client = bigquery.Client()

    #query = """
        #SELECT *
       # FROM f`ny_rides_all.{table_name}`
    #"""

    #client.query(query)  # Make an API request.

In [11]:
# services = ['fhv','green','yellow']
init_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/'
#BUCKET = os.environ.get("GCP_GCS_BUCKET")

#web_to_gcs('2019', 'green')

#web_to_gcs('2020', 'green')

#web_to_gcs('2019', 'yellow')

#web_to_gcs('2020', 'yellow')

web_to_gcs('2019', 'fhv')

PipelineStepFailed: Pipeline execution failed at stage sync with exception:

<class 'dlt.common.configuration.exceptions.ConfigFieldMissingException'>
Following fields are missing: ['project_id', 'private_key', 'client_email'] in configuration with spec GcpServiceAccountCredentials
	for field "project_id" config providers and keys were tried in following order:
		In Environment Variables key DLT_IPYKERNEL_LAUNCHER__DESTINATION__BIGQUERY__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DLT_IPYKERNEL_LAUNCHER__DESTINATION__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DLT_IPYKERNEL_LAUNCHER__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DESTINATION__BIGQUERY__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key DESTINATION__CREDENTIALS__PROJECT_ID was not found.
		In Environment Variables key CREDENTIALS__PROJECT_ID was not found.
	for field "private_key" config providers and keys were tried in followi